In [21]:
pip install lifetimes

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install skimpy

Note: you may need to restart the kernel to use updated packages.


In [23]:
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import datetime as dt
pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)
pd.set_option("display.float_format",lambda x: "%3f" % x)
from skimpy import skim

In [24]:

###############################################################
# GÖREVLER
###############################################################
# GÖREV 1: Veriyi Hazırlama
           # 1. flo_data_20K.csv verisini okuyunuz.Dataframe’in kopyasını oluşturunuz.


data=pd.read_csv("/kaggle/input/flo-dataset/flo_data.csv")
datacopy=data.copy()
datacopy.head()
datacopy.describe().T
datacopy.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [25]:
skim(datacopy)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 19945  │ │ string      │ 8     │                                                          │
│ │ Number of columns │ 12     │ │ float64     │ 4     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name             ┃ NA  ┃ NA %  ┃ mean   ┃ sd     ┃ p0     ┃ p25    ┃ p50   ┃ p75   ┃ p100  ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━┩  │
│ │ order_num_total_ever_on │   0 │     0 │  3.111 │  4.226 │      1 │      1 │     2 │     4 │   200 │   ▇    │  │
│ │ line                    │     │       │        │        │        │        │       │       │       │        │  │
│ │ order_num_total_ever_of │   0 │     0 │  1.914 │  2.063 │      1 │      1 │     1 │     2 │   109 │   ▇    │  │
│ │ fline                   │     │       │        │        │        │        │       │       │       │        │  │
│ │ customer_value_total_ev │   0 │     0 │  253.9 │  301.5 │     10 │  99.99 │   180 │   320 │ 18120 │   ▇    │  │
│ │ er_offline              │     │       │        │        │        │        │       │       │       │        │  │
│ │ customer_value_total_ev │   0 │     0 │  497.3 │  832.6 │  12.99 │    150 │ 286.5 │ 578.4 │ 45220 │   ▇    │  │
│ │ er_online               │     │       │        │        │        │        │       │       │       │        │  │
│ └─────────────────────────┴─────┴───────┴────────┴────────┴────────┴────────┴───────┴───────┴───────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                           ┃ NA    ┃ NA %      ┃ words per row            ┃ total words         ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ master_id                             │     0 │         0 │                        1 │               19945 │  │
│ │ order_channel                         │     0 │         0 │                      1.6 │               32273 │  │
│ │ last_order_channel                    │     0 │         0 │                      1.4 │               28424 │  │
│ │ first_order_date                      │     0 │         0 │                        1 │               19945 │  │
│ │ last_order_date                       │     0 │         0 │                        1 │               19945 │  │
│ │ last_order_date_online                │     0 │         0 │                        1 │               19945 │  │
│ │ last_order_date_offline               │     0 │         0 │                        1 │               19945 │  │
│ │ interested_in_categories_12           │     0 │         0 │                      1.7 │               33178 │  │
│ └───────────────────────────────────────┴───────┴───────────┴──────────────────────────┴─────────────────────┘  │
╰────────────────────────────────────────────────────── 

In [26]:

# 2. Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds fonksiyonlarını tanımlayınız.
           # Not: cltv hesaplanırken frequency değerleri integer olması gerekmektedir.Bu nedenle alt ve üst limitlerini round() ile yuvarlayınız.
def outlier_threshold(dataframe,variable):
    quartile1=dataframe[variable].quantile(0.01)
    quartile3=dataframe[variable].quantile(0.99)
    interquantile_range=quartile3-quartile1
    up_limit=(quartile3+1.5 * interquantile_range)
    low_limit=quartile1 + 1.5 * interquantile_range
    return low_limit,up_limit

def replace_with_threshold(dataframe,variable):
    low_limit,up_limit=outlier_threshold(dataframe,variable)
    #dataframe.loc[(dataframe[variable] < low_limit), variable]= low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable]= up_limit
    if variable == 'frequency':  # frequency'nin yuvarlanması
        dataframe[variable] = dataframe[variable].round()

In [27]:

# 3. "order_num_total_ever_online","order_num_total_ever_offline","customer_value_total_ever_offline",
# "customer_value_total_ever_online" değişkenlerinin
# aykırı değerleri varsa baskılayanız.

replace_with_threshold(datacopy,"order_num_total_ever_online")
replace_with_threshold(datacopy,"order_num_total_ever_offline")
replace_with_threshold(datacopy,"customer_value_total_ever_offline")
replace_with_threshold(datacopy,"customer_value_total_ever_online")
datacopy.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000000,3.092304,3.813699,1.000000,1.000000,2.000000,4.000000,48.500000
order_num_total_ever_offline,19945.000000,1.886187,1.434694,1.000000,1.000000,1.000000,2.000000,16.000000
customer_value_total_ever_offline,19945.000000,251.921220,251.022560,10.000000,99.990000,179.980000,319.970000,3019.882000
customer_value_total_ever_online,19945.000000,489.705377,632.606387,12.990000,149.980000,286.460000,578.440000,7799.541000


In [28]:

# 4. Omnichannel müşterilerin hem online'dan hemde offline platformlardan alışveriş yaptığını ifade etmektedir. Herbir müşterinin toplam
# alışveriş sayısı ve harcaması için yeni değişkenler oluşturun.


datacopy["Omnichannel_Counts"]=datacopy["order_num_total_ever_online"]+datacopy["order_num_total_ever_offline"]
datacopy["Omnichannel_Price"]=datacopy["customer_value_total_ever_offline"]+datacopy["customer_value_total_ever_online"]

In [29]:

for i in datacopy.columns:
    if "date" in i:
        datacopy[i]=pd.to_datetime(datacopy[i])



datacopy.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [30]:

# GÖREV 2: CLTV Veri Yapısının Oluşturulması
# 1.Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız.

print(datacopy["last_order_date"].max())
today_date=dt.datetime(2021,6,1)

2021-05-30 00:00:00


CLTV --> Bir müşterinin bir şirketle kurduğu ilişki-iletişim süresince bu şirkete kazandırdığı parasal değerdir.

CLTV= (Customer Value/ Churn Rate)*Profit Margin  formülü ile hesaplanır.


In [31]:

# 2.customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı yeni bir cltv dataframe'i oluşturunuz.

cltv_df=pd.DataFrame()
cltv_df["master_id"]=datacopy["master_id"]
cltv_df["recency_cltv_weekly"]=(datacopy["last_order_date"]-datacopy["first_order_date"]).dt.days
cltv_df["T_weekly"]=(today_date-datacopy["first_order_date"]).dt.days
cltv_df["frequency"]=datacopy["Omnichannel_Counts"]
cltv_df["monetary_cltv_avg"]=datacopy["Omnichannel_Price"]

cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,119,214,5.000000,939.370000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,1469,1574,21.000000,2013.550000
2,69b69676-1a40-11ea-941b-000d3a38a36f,366,552,5.000000,585.320000
3,1854e56c-491f-11eb-806e-000d3a38a36f,11,146,2.000000,121.970000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,582,668,2.000000,209.980000


RFM analizde recency değeri günlük olarak hesaplanırken CLTV analizinde haftalık olarak hesaplanır.

**Recency** = Son satın alma üzerinden geçen zaman. Son tarih - ilk tarih.                             
**T**: Müşterinin yaşı. Haftalık ( analiz tarihi ile ilk satın aldığı tarih arasındaki fark)         
**frequency **= tekrar eden  toplam satın alma sayısı, 1 den büyük olmalıdır. En az 2 kere gelmesi o firmanın müşterisi kabul edilir.                                                                    
**monatary_value** = satın alma başına ortalama kazanç. Fiyatın satın alma sayısına bölünmesi ile bulunur.Rfm analizinde direkt olarak fiyatı baz alıyoruzç

In [32]:

# Monetary değeri satın alma başına ortalama değer olarak, recency ve tenure değerleri ise haftalık cinsten ifade edilecek.

cltv_df["recency_cltv_weekly"]=cltv_df["recency_cltv_weekly"]/7
cltv_df["T_weekly"]=cltv_df["T_weekly"]/7
cltv_df["monetart_cltv_avg"]=cltv_df["monetary_cltv_avg"]/cltv_df["frequency"]
cltv_df=cltv_df[cltv_df["frequency"] >1 ]
cltv_df.head()


,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.000000,939.370000,187.874000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.000000,2013.550000,95.883333
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.000000,585.320000,117.064000
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.000000,121.970000,60.985000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.000000,209.980000,104.990000


BG/NBD --> Olasılıksal bir modeldir.2 süreci olasılıksal olarak modeller.
Transaction Process(Buy) + Dropout Process(Till you die)

Transaction Process --> Satın alma işlemi sürecidir.Bir müşteri canlı olduğu sürece kendi transaction rate'si etrafında satın alma yapmaya devam edecektir.

Dropout Process --> Bir müşteri alışveriş yaptıktan sonra belirli bir olasılıkla yok olur.



CLTV= BG/NBD MOEL * GAMMA GAMMA SUBMODEL


In [33]:
# GÖREV 3: BG/NBD, Gamma-Gamma Modellerinin Kurulması, CLTV'nin hesaplanması
           # 1. BG/NBD modelini fit ediniz.
bgf=BetaGeoFitter(penalizer_coef=0.001) ##penalizer_coef --> ceza katsayısıdır.  Beta ve gama parametrelerini bulur.
bgf.fit(cltv_df["frequency"],
        cltv_df["recency_cltv_weekly"],
        cltv_df["T_weekly"])

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [34]:

# a. 3 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_3_month olarak cltv dataframe'ine ekleyiniz.

cltv_df["exp_sales_3_months"]=bgf.conditional_expected_number_of_purchases_up_to_time(4*3, ## Bir ay 4 hafta olduğu için.
                                                        cltv_df["frequency"],
                                                        cltv_df["recency_cltv_weekly"], # Bu değerlerimiz haftalık olduğu için 4*3 yaparız.
                                                        cltv_df["T_weekly"])
cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.000000,939.370000,187.874000,0.973927
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.000000,2013.550000,95.883333,0.983161
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.000000,585.320000,117.064000,0.670586
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.000000,121.970000,60.985000,0.700412
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.000000,209.980000,104.990000,0.396039


In [35]:

# b. 6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_6_month olarak cltv dataframe'ine ekleyiniz.
# en çok etkileyen etken frequency.
cltv_df["exp_Sales_6_month"]=bgf.conditional_expected_number_of_purchases_up_to_time(4*6,
                                                                                     cltv_df["frequency"],
                                                                                     cltv_df["recency_cltv_weekly"],
                                                                                     cltv_df["T_weekly"])

cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.000000,939.370000,187.874000,0.973927,1.947853
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.000000,2013.550000,95.883333,0.983161,1.966323
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.000000,585.320000,117.064000,0.670586,1.341172
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.000000,121.970000,60.985000,0.700412,1.400824
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.000000,209.980000,104.990000,0.396039,0.792077


**GAMMA GAMMA SUBMODEL-->** Bir müşterinin işlem başına ortalama ne kadar kar getirebileceğini tahmin etmek için kuklanılır. Bu  yüzden frequency ve monetary üzerine odaklanılır.


In [36]:
#2. Gamma-Gamma modelini fit ediniz. Müşterilerin ortalama bırakacakları değeri tahminleyip exp_average_value olarak cltv dataframe'ine ekleyiniz.
replace_with_threshold(cltv_df,"frequency")
# cltv_df["frequency"]=cltv_df["frequency"].astype(int)
ggf=GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df["frequency"],cltv_df["monetary_cltv_avg"])
cltv_df["exp_avg_value"]=ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                                                           cltv_df["monetary_cltv_avg"])


In [37]:
cltv_df

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month,exp_avg_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.000000,939.370000,187.874000,0.973927,1.947853,979.840452
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.000000,2013.550000,95.883333,0.983161,1.966323,2033.366678
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.000000,585.320000,117.064000,0.670586,1.341172,610.812677
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.000000,121.970000,60.985000,0.700412,1.400824,137.695533
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.000000,209.980000,104.990000,0.396039,0.792077,235.644166
...,...,...,...,...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,41.142857,88.428571,3.000000,401.960000,133.986667,0.485785,0.971569,432.377511
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,42.285714,65.285714,2.000000,390.470000,195.235000,0.480429,0.960859,436.516212
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,88.714286,89.857143,3.000000,632.940000,210.980000,0.481605,0.963210,680.115819
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,98.428571,113.857143,6.000000,1009.770000,168.295000,0.610224,1.220448,1045.723720


In [38]:

# 3. 6 aylık CLTV hesaplayınız ve cltv ismiyle dataframe'e ekleyiniz.


cltv_df["cltv"]=ggf.customer_lifetime_value(bgf,
                                 cltv_df["frequency"],
                                 cltv_df["recency_cltv_weekly"],
                                 cltv_df["T_weekly"],
                                 cltv_df["monetary_cltv_avg"],
                                 time=6*4, ## Değerlerimiz haftalık olduğu için ve bir ay dört hafta içerdiği için 
                                 freq="W",
                                 discount_rate=0.01  # indirm oranı.Bazı satışlarda indirim yapılmış olabilir.
                                 )
cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month,exp_avg_value,cltv
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.000000,939.370000,187.874000,0.973927,1.947853,979.840452,7340.300758
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.000000,2013.550000,95.883333,0.983161,1.966323,2033.366678,15377.043734
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.000000,585.320000,117.064000,0.670586,1.341172,610.812677,3150.611024
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.000000,121.970000,60.985000,0.700412,1.400824,137.695533,741.832040
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.000000,209.980000,104.990000,0.396039,0.792077,235.644166,717.838239


In [39]:
                              
# b. Cltv değeri en yüksek 20 kişiyi gözlemleyiniz.
cltv_df.sort_values(by="cltv",ascending=False).head(20)



,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month,exp_avg_value,cltv
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714286,67.285714,52.000000,8643.221000,164.632781,4.697962,9.395924,8677.151978,310767.333528
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.142857,147.142857,50.000000,7930.031000,160.202646,2.856772,5.713544,7962.418319,176611.213847
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.571429,58.714286,31.000000,7084.430000,228.530000,3.083779,6.167558,7131.226371,169153.026942
14181,0e8c7b30-ac18-11e9-a2fc-000d3a38a36f,141.714286,159.714286,48.000000,8180.731000,170.431896,2.628214,5.256428,8215.537936,166084.478474
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.142857,111.857143,43.000000,6755.840000,157.112558,2.978047,5.956093,6787.956882,155490.140120
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.571429,113.428571,40.000000,7048.000000,176.200000,2.763492,5.526983,7084.027731,150581.189903
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,204.857143,211.857143,50.000000,7882.021000,159.232747,2.214914,4.429828,7914.212666,136101.322027
14330,90d628b6-1928-11ea-b6a2-000d3a38a36f,98.571429,117.428571,32.000000,7879.521000,246.235031,2.210530,4.421059,7929.920091,134833.393455
4157,7eed6468-4540-11ea-acaf-000d3a38a36f,89.142857,90.000000,27.000000,7823.630000,289.764074,2.214331,4.428662,7883.009671,134266.262454
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.142857,13.142857,11.000000,8338.481000,758.043727,1.970108,3.940216,8495.531325,128739.802938


In [40]:
## segmentlere ayırınız.
cltv_df["segment"]=pd.qcut(cltv_df["cltv"],q=4,labels=["D","C","B","A"])
cltv_df.head()

,master_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,monetart_cltv_avg,exp_sales_3_months,exp_Sales_6_month,exp_avg_value,cltv,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.000000,939.370000,187.874000,0.973927,1.947853,979.840452,7340.300758,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.000000,2013.550000,95.883333,0.983161,1.966323,2033.366678,15377.043734,A
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.000000,585.320000,117.064000,0.670586,1.341172,610.812677,3150.611024,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.000000,121.970000,60.985000,0.700412,1.400824,137.695533,741.832040,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.000000,209.980000,104.990000,0.396039,0.792077,235.644166,717.838239,D
